In [1]:
# Import Necessary Libraries 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [12]:
# Load and Explore Dataset
# Load the dataset
data = pd.read_csv('data/final_merged_data.csv')

# Explore the dataset
data.head()

,User,Card,Month,Day,Time,Use Chip,Merchant City,MCC,Errors?,Is Fraud?,...,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web
0,0,0,9,1,06:21,Swipe Transaction,La Verne,5300,NaN,No,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,0,0,9,1,06:42,Swipe Transaction,Monterey Park,5411,NaN,No,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
2,0,0,9,2,06:22,Swipe Transaction,Monterey Park,5411,NaN,No,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
3,0,0,9,2,17:45,Swipe Transaction,Monterey Park,5651,NaN,No,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
4,0,0,9,3,06:23,Swipe Transaction,La Verne,5912,NaN,No,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No


In [13]:
# Explore the dataset
data.head()

,User,Card,Month,Day,Time,Use Chip,Merchant City,MCC,Errors?,Is Fraud?,...,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web
0,0,0,9,1,06:21,Swipe Transaction,La Verne,5300,NaN,No,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,0,0,9,1,06:42,Swipe Transaction,Monterey Park,5411,NaN,No,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
2,0,0,9,2,06:22,Swipe Transaction,Monterey Park,5411,NaN,No,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
3,0,0,9,2,17:45,Swipe Transaction,Monterey Park,5651,NaN,No,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
4,0,0,9,3,06:23,Swipe Transaction,La Verne,5912,NaN,No,...,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5927394 entries, 0 to 5927393
Data columns (total 30 columns):
 #   Column                       Dtype 
---  ------                       ----- 
 0   User                         int64 
 1   Card                         int64 
 2   Month                        int64 
 3   Day                          int64 
 4   Time                         object
 5   Use Chip                     object
 6   Merchant City                object
 7   MCC                          int64 
 8   Errors?                      object
 9   Is Fraud?                    object
 10  Current Age                  int64 
 11  City                         object
 12  Zipcode                      int64 
 13  Per Capita Income - Zipcode  object
 14  Yearly Income - Person       object
 15  Total Debt                   object
 16  FICO Score                   int64 
 17  Num Credit Cards             int64 
 18  CARD INDEX                   int64 
 19  Card Brand           

In [15]:
data.describe()

,User,Card,Month,Day,MCC,Current Age,Zipcode,FICO Score,Num Credit Cards,CARD INDEX,Card Number,CVV,Cards Issued,Year PIN last Changed
count,5.927394e+06,5.927394e+06,5.927394e+06,5.927394e+06,5.927394e+06,5.927394e+06,5.927394e+06,5.927394e+06,5.927394e+06,5.927394e+06,5.927394e+06,5.927394e+06,5.927394e+06,5.927394e+06
mean,2.471879e+02,1.334030e+00,6.521481e+00,1.571408e+01,5.554698e+03,5.370343e+01,5.391524e+04,7.092009e+02,3.646606e+00,1.334030e+00,4.870725e+15,5.001100e+02,1.544360e+00,2.011123e+03
std,1.471788e+02,1.399626e+00,3.472707e+00,8.791528e+00,8.760748e+02,1.618908e+01,2.903358e+04,6.712847e+01,1.566562e+00,1.399626e+00,1.232094e+15,2.911622e+02,5.103461e-01,2.906984e+00
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.711000e+03,1.800000e+01,1.524000e+03,4.880000e+02,1.000000e+00,0.000000e+00,3.001055e+14,0.000000e+00,1.000000e+00,2.003000e+03
25%,1.220000e+02,0.000000e+00,3.000000e+00,8.000000e+00,5.300000e+03,4.100000e+01,3.173000e+04,6.810000e+02,3.000000e+00,0.000000e+00,4.560018e+15,2.580000e+02,1.000000e+00,2.009000e+03
50%,2.490000e+02,1.000000e+00,7.000000e+00,1.600000e+01,5.499000e+03,5.200000e+01,4.904800e+04,7.120000e+02,4.000000e+00,1.000000e+00,5.116904e+15,5.130000e+02,2.000000e+00,2.011000e+03
75%,3.710000e+02,2.000000e+00,1.000000e+01,2.300000e+01,5.812000e+03,6.300000e+01,7.862900e+04,7.490000e+02,5.000000e+00,2.000000e+00,5.569049e+15,7.410000e+02,2.000000e+00,2.013000e+03
max,4.990000e+02,8.000000e+00,1.200000e+01,3.100000e+01,9.402000e+03,1.010000e+02,9.950800e+04,8.500000e+02,9.000000e+00,8.000000e+00,6.997197e+15,9.990000e+02,3.000000e+00,2.020000e+03


In [16]:
data.isnull().sum()

User                                 0
Card                                 0
Month                                0
Day                                  0
Time                                 0
Use Chip                             0
Merchant City                        0
MCC                                  0
Errors?                        5831186
Is Fraud?                            0
Current Age                          0
City                                 0
Zipcode                              0
Per Capita Income - Zipcode          0
Yearly Income - Person               0
Total Debt                           0
FICO Score                           0
Num Credit Cards                     0
CARD INDEX                           0
Card Brand                           0
Card Type                            0
Card Number                          0
Expires                              0
CVV                                  0
Has Chip                             0
Cards Issued             

In [17]:
data.shape

(5927394, 30)

In [8]:
#data_dropna = data.dropna()

In [9]:
#data_dropna.shape

(96208, 30)

In [20]:
# Define the target variable
target = 'Is Fraud?'
y = data[target]

In [21]:
# Define features
features = data.columns.drop(target)
X = data[features]

In [ ]:
# Clean numerical columns by removing '$' and ',' from the 'Per Capita Income - Zipcode' and 'Yearly Income - Person' columns and convert them to float
X.loc[:, 'Per Capita Income - Zipcode'] = X['Per Capita Income - Zipcode'].replace('[\$,]', '', regex=True).astype(float)
X.loc[:, 'Yearly Income - Person'] = X['Yearly Income - Person'].replace('[\$,]', '', regex=True).astype(float)


In [22]:
# Clean numerical columns by removing '$' and ',' from the 'Per Capita Income - Zipcode' and 'Yearly Income - Person' columns and convert them to float
X['Per Capita Income - Zipcode'] = X['Per Capita Income - Zipcode'].replace('[\$,]', '', regex=True).astype(float)
X['Yearly Income - Person'] = X['Yearly Income - Person'].replace('[\$,]', '', regex=True).astype(float)


C:\Users\kjyon\AppData\Local\Temp\ipykernel_20324\25072413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Per Capita Income - Zipcode'] = X['Per Capita Income - Zipcode'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\kjyon\AppData\Local\Temp\ipykernel_20324\25072413.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Yearly Income - Person'] = X['Yearly Income - Person'].replace('[\$,]', '', regex=True).astype(float)


In [ ]:
# Define categorical and numerical columns
categorical_cols = ['User', 'Card', 'Month', 'Day', 'Time', 'Use Chip', 'Merchant City', 
                    'Errors?', 'City', 'Age Group', 'Card Brand', 'Card Type', 'Has Chip', 
                    'Card on Dark Web']
numerical_cols = ['MCC', 'Zipcode', 'Per Capita Income - Zipcode', 'Yearly Income - Person', 
                  'Total Debt', 'FICO Score', 'Num Credit Cards', 'CARD INDEX', 'Cards Issued', 
                  'Credit Limit', 'Year PIN last Changed']

In [ ]:
# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

In [ ]:
# Define the model 1 - option 1
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

In [ ]:
# Define the model - option 2
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, max_features='auto', max_depth=10, criterion='gini', random_state=42))
])

In [ ]:
# Define the model - option 3
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1, random_state=42))
])

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Model evaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')